In [69]:
MAX_RANGE = 426 #Maximum Tesla Model S range in Km from https://en.wikipedia.org/wiki/Tesla_Model_S
import geohash
import pandas as pd

df = pd.read_csv("Teslarati_SC_data.csv")
df["lat"], df["lon"] = zip(*df["GPS"].str.split(',').tolist())
df["lat"], df["lon"] = df["lat"].astype(float), df["lon"].astype(float)
df['geohash'] = df.apply(lambda x: geohash.encode(x['lat'],x['lon']), axis=1)

In [44]:
BASE_32 = "0123456789bcdefghjkmnpqrstuvwxyz"

NEIGHBORS = {'right':{'even': "bc01fg45238967deuvhjyznpkmstqrwx",'odd':"238967debc01fg45kmstqrwxuvhjyznp"},
             'left':{'even':"238967debc01fg45kmstqrwxuvhjyznp",'odd':"14365h7k9dcfesgujnmqp0r2twvyx8zb"},
             'top':{'even':"p0r21436x8zb9dcf5h7kjnmqesgutwvy",'odd':"bc01fg45238967deuvhjyznpkmstqrwx"},
             'bottom':{'even':"14365h7k9dcfesgujnmqp0r2twvyx8zb",'odd':"238967debc01fg45kmstqrwxuvhjyznp"}
            }
             
BORDERS = {'right':{'even':"bcfguvyz",'odd':"prxz"},
           'left':{'even':"0145hjnp",'odd':"028b"},
           'top':{'even':"prxz",'odd':"bcfguvyz"},
           'bottom':{'even':"028b",'odd':"0145hjnp"}
          }

def calc_adj_geohash(src_hash, search_direction):
    L_src_hash = src_hash.lower()
    last_char = L_src_hash[-1]
    hash_type = odd_even(L_src_hash)
    base = L_src_hash[0:len(L_src_hash)-1]
    if  last_char in BORDERS[search_direction][hash_type]:
        base = calc_adj_geohash(base, search_direction)
    return base + BASE_32[NEIGHBORS[search_direction][hash_type].index(last_char)]

test_adj = calc_adj_geohash(df['geohash'][0][0:3],'bottom')
print(geohash.decode(df['geohash'][0][0:3]))
print (geohash.decode(test_adj))

(37.265625, -117.421875)
(35.859375, -117.421875)


In [47]:
def get_adj_geohashes(src_hash):
    directions = ['right','left','top','bottom']
    adj_hash_dict = {direction:calc_adj_geohash(src_hash,direction) for direction in directions}
    adj_hash_dict['top_right'] = calc_adj_geohash(adj_hash_dict['right'],'top')
    adj_hash_dict['bottom_right'] = calc_adj_geohash(adj_hash_dict['right'],'bottom')
    adj_hash_dict['top_left'] = calc_adj_geohash(adj_hash_dict['left'],'top')
    adj_hash_dict['bottom_left'] = calc_adj_geohash(adj_hash_dict['left'],'bottom')
    adj_hash_dict['middle'] = src_hash
    return adj_hash_dict    

In [4]:
def odd_even(str_hash):
    if len(str_hash)%2 == 0:
        return 'even'
    else:
        return 'odd'

In [70]:
x = [gh for gh in df['geohash'] if gh[0:3] in list(get_adj_geohashes(df['geohash'][0][0:3]).values())]
x

['9qsczpyjczz0',
 '9q717h3ht9ye',
 '9qeqhugxq5w5',
 '9qk4z15rwvbw',
 '9qs01qvvvuqz',
 '9qhcj9n0pdzu',
 '9qh6jd1wn67n',
 '9qhwr8hfzk7x',
 '9qh0ju82wcpg',
 '9qu8s66h4rhe']

In [76]:
import configparser
import googlemaps

def get_geohash_distance(gh_A,gh_B):
    config = configparser.ConfigParser()
    config.read("config.ini")
    API_key = config['Keys']['google_API']
    gmaps = googlemaps.Client(key=API_key)
    
    GPS_A = geohash.decode(gh_A)
    GPS_B = geohash.decode(gh_B)
    
    directions_result = gmaps.directions(GPS_A,
                                     GPS_B,
                                     mode="driving")
    return directions_result[0]['legs'][0]['distance']['value'] 

In [77]:
get_geohash_distance('9qsczpyjczz0','9q717h3ht9ye')

416278

In [89]:
GEOHASH_PRECISION = 3
def build_node_distance_list(src_hash):
    node_hashes = ([gh for gh in df['geohash']
                  if gh[0:GEOHASH_PRECISION] in list(get_adj_geohashes(src_hash[0:GEOHASH_PRECISION]).values())
                 and gh != src_hash])
    node_tuple = [(node_gh,get_geohash_distance(src_hash,node_gh)) for node_gh in node_hashes]
    return ([(node[0],node[1]) for node in node_tuple if node[1]/1000 <= MAX_RANGE])

In [90]:
print (build_node_distance_list('9qh0ju82wcpg'))

[('9q717h3ht9ye', 246097), ('9q5f7vrfnun9', 67507), ('9mupsr6uztt1', 19929), ('9muejjwkcfbu', 126716), ('9q5zwb5ckgcm', 204636), ('9qk4z15rwvbw', 262261), ('9qs01qvvvuqz', 372753), ('9q5bf6r884m5', 53221), ('9q5c3gngk4fc', 65057), ('9qhcj9n0pdzu', 116127), ('9qh6jd1wn67n', 69371), ('9q56b0cwt7en', 146851), ('9muqdd0q8tpj', 39873), ('9qhwr8hfzk7x', 171401), ('9q5bgr4stcks', 54387), ('9q5rj33bpbys', 185376)]
